In [1]:
!pip install transformers
!pip install torch
!pip install pandas
!pip install sklearn


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
!pip install scikit-learn

In [3]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import BertConfig
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder


In [5]:
# Load the data
file1 = pd.read_csv('/content/sample_data/files/all_risk_categories.csv')
file2 = pd.read_csv('/content/sample_data/files/test.csv')

# Encode the categories
label_encoder = LabelEncoder()
file1['label'] = label_encoder.fit_transform(file1['category'])

# Split the data for training and validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    file1['Item1A'].tolist(), file1['label'].tolist(), test_size=0.2, random_state=42
)


In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=256)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

ValueError: Input is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.

In [7]:
print(type(train_texts))
print(type(val_texts))

# Ensure each element is a string
print(type(train_texts[0]))
print(type(val_texts[0]))


<class 'list'>
<class 'list'>
<class 'str'>
<class 'str'>


In [8]:
# Print a few examples from train_texts and val_texts
print("Sample train_texts:", train_texts[:5])
print("Sample val_texts:", val_texts[:5])


Sample train_texts: ["[O]ur or our customers' sensitive, proprietary, or confidential information could be leaked, disclosed, orrevealed as a result of or in connection with our employees', personnel's, or vendors' use of generative AItechnologies. Any such information that we input into a third-party generative AI or machine learning (“ML”) platform could be revealed to others, including if information is used to train the third party's\nAI/ML models. Additionally, where an AI/ML model ingests personal information\nand makes connections\nusing such data, those technologies may reveal other sensitive, proprietary, or confidential information\ngenerated by the model. Moreover, AI/ML models may create incomplete, inaccurate, or otherwise flawed\noutputs, some of which may\nappear correct. We may use AI/ML outputs to make certain decisions. Due to\nthese potential flaws, the model could lead us to make decisions that could bias certain individuals or classes\nof individuals and adversely 

In [10]:
# Replace newline characters with a space to preserve word boundaries
train_texts = [text.replace('\n', ' ') for text in train_texts]
val_texts = [text.replace('\n', ' ') for text in val_texts]

# Print cleaned sample data
print("Cleaned train_texts:", train_texts[:5])
print("Cleaned val_texts:", val_texts[:5])


AttributeError: 'float' object has no attribute 'replace'

In [11]:
# Replace non-string values with empty strings, then replace newlines and trim spaces
train_texts_cleaned = [str(text).replace('\n', ' ').strip() if isinstance(text, str) else "" for text in train_texts]
val_texts_cleaned = [str(text).replace('\n', ' ').strip() if isinstance(text, str) else "" for text in val_texts]

# Print cleaned sample data
print("Cleaned Sample train_texts:", train_texts_cleaned[:5])
print("Cleaned Sample val_texts:", val_texts_cleaned[:5])

# Retry tokenization with cleaned data
try:
    train_encodings = tokenizer(train_texts_cleaned, truncation=True, padding=True, max_length=256)
    val_encodings = tokenizer(val_texts_cleaned, truncation=True, padding=True, max_length=256)
    print("Tokenization successful!")
except ValueError as e:
    print(f"Tokenization failed: {e}")


Cleaned Sample train_texts: ["[O]ur or our customers' sensitive, proprietary, or confidential information could be leaked, disclosed, orrevealed as a result of or in connection with our employees', personnel's, or vendors' use of generative AItechnologies. Any such information that we input into a third-party generative AI or machine learning (“ML”) platform could be revealed to others, including if information is used to train the third party's AI/ML models. Additionally, where an AI/ML model ingests personal information and makes connections using such data, those technologies may reveal other sensitive, proprietary, or confidential information generated by the model. Moreover, AI/ML models may create incomplete, inaccurate, or otherwise flawed outputs, some of which may appear correct. We may use AI/ML outputs to make certain decisions. Due to these potential flaws, the model could lead us to make decisions that could bias certain individuals or classes of individuals and adversely 

In [13]:
# Print a few examples from train_texts and val_texts
print("Sample train_texts:", train_texts[:5])
print("Sample val_texts:", val_texts[:5])


Sample train_texts: ["[O]ur or our customers' sensitive, proprietary, or confidential information could be leaked, disclosed, orrevealed as a result of or in connection with our employees', personnel's, or vendors' use of generative AItechnologies. Any such information that we input into a third-party generative AI or machine learning (“ML”) platform could be revealed to others, including if information is used to train the third party's\nAI/ML models. Additionally, where an AI/ML model ingests personal information\nand makes connections\nusing such data, those technologies may reveal other sensitive, proprietary, or confidential information\ngenerated by the model. Moreover, AI/ML models may create incomplete, inaccurate, or otherwise flawed\noutputs, some of which may\nappear correct. We may use AI/ML outputs to make certain decisions. Due to\nthese potential flaws, the model could lead us to make decisions that could bias certain individuals or classes\nof individuals and adversely 

In [14]:
import re

def clean_text(text):
    # Replace newline characters with spaces
    text = text.replace('\n', ' ')
    # Remove special characters like brackets
    text = re.sub(r'[^\w\s]', '', text)
    # Remove excessive spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply the clean_text function to train_texts and val_texts
train_texts_cleaned = [clean_text(str(text)) if isinstance(text, str) else "" for text in train_texts]
val_texts_cleaned = [clean_text(str(text)) if isinstance(text, str) else "" for text in val_texts]

# Print cleaned sample data
print("Cleaned Sample train_texts:", train_texts_cleaned[:5])
print("Cleaned Sample val_texts:", val_texts_cleaned[:5])

# Retry tokenization with cleaned data
try:
    train_encodings = tokenizer(train_texts_cleaned, truncation=True, padding=True, max_length=256)
    val_encodings = tokenizer(val_texts_cleaned, truncation=True, padding=True, max_length=256)
    print("Tokenization successful!")
except ValueError as e:
    print(f"Tokenization failed: {e}")


Cleaned Sample train_texts: ['Our or our customers sensitive proprietary or confidential information could be leaked disclosed orrevealed as a result of or in connection with our employees personnels or vendors use of generative AItechnologies Any such information that we input into a thirdparty generative AI or machine learning ML platform could be revealed to others including if information is used to train the third partys AIML models Additionally where an AIML model ingests personal information and makes connections using such data those technologies may reveal other sensitive proprietary or confidential information generated by the model Moreover AIML models may create incomplete inaccurate or otherwise flawed outputs some of which may appear correct We may use AIML outputs to make certain decisions Due to these potential flaws the model could lead us to make decisions that could bias certain individuals or classes of individuals and adversely impact their rights As a result we co

In [16]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts_cleaned, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_texts_cleaned, truncation=True, padding=True, max_length=256)


In [17]:
class RiskDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = RiskDataset(train_encodings, train_labels)
val_dataset = RiskDataset(val_encodings, val_labels)


In [18]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy="steps",     # Evaluate every `logging_steps`
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Start training
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss


TrainOutput(global_step=6, training_loss=2.0789305369059243, metrics={'train_runtime': 101.2989, 'train_samples_per_second': 0.355, 'train_steps_per_second': 0.059, 'total_flos': 3811170279600.0, 'train_loss': 2.0789305369059243, 'epoch': 3.0})

In [20]:
# Tokenize test data
test_texts = file2['Item1A'].tolist()
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=256)
test_dataset = RiskDataset(test_encodings, [0]*len(test_texts))  # Dummy labels

# Predict the categories
predictions = trainer.predict(test_dataset)
predicted_labels = torch.argmax(predictions.predictions, dim=1)

# Decode the labels
predicted_categories = label_encoder.inverse_transform(predicted_labels.numpy())

# Save the output to a new CSV
output = pd.DataFrame({'Item1A': file2['Item1A'], 'category': predicted_categories})
output.to_csv('output.csv', index=False)


TypeError: argmax(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

In [23]:
# Tokenize test data
test_texts = file2['Item1A'].tolist()
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=256)
test_dataset = RiskDataset(test_encodings, [0]*len(test_texts))  # Dummy labels

# Predict the categories
predictions = trainer.predict(test_dataset)
predicted_labels = torch.argmax(torch.tensor(predictions.predictions), dim=1)

# Decode the labels
predicted_categories = label_encoder.inverse_transform(predicted_labels.numpy())

# Save the output to a new CSV
output = pd.DataFrame({'Item1A': file2['Item1A'], 'category': predicted_categories})
output.to_csv('output.csv', index=False)


In [24]:
model.save_pretrained('./saved_model')
tokenizer.save_pretrained('./saved_model')


('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.txt',
 './saved_model/added_tokens.json')

In [25]:
df1 = pd.read_csv('/content/output.csv')
df1.head()

,Item1A,category
0,. Risk Factors Our operations and financial re...,Ethical Risk
1,". Risk Factors. Our business, prospects, finan...",Ethical Risk
2,. Risk Factors RISK FACTORS A description of t...,Ethical Risk
3,. Risk Factors. An investment in our securitie...,Competition Risk
4,. Ri sk Factors FORWARD-LOOKING STATEMENTS Thi...,Ethical Risk


In [33]:
# Create the output DataFrame with filename, Item1A, and category columns
output = pd.DataFrame({
    'filename': file2['filename'],
    'Item1A': file2['Item1A'],
    'category': predicted_categories
})

# Save the output to a new JSON
output.to_json('output.json', orient ='records',lines = True)
print("File saved!")


File saved!


In [28]:
df2 = pd.read_json('/content/output.json', orient='records', lines=True)
df2.head()

,filename,Item1A,category
0,20230301_10-K_edgar_data_1831915_0000950170-23...,. Risk Factors Our operations and financial re...,Ethical Risk
1,20230221_10-K_edgar_data_1410384_0001410384-23...,". Risk Factors. Our business, prospects, finan...",Ethical Risk
2,20230228_10-K_edgar_data_1315098_0001315098-23...,. Risk Factors RISK FACTORS A description of t...,Ethical Risk
3,20230113_10-K_edgar_data_1829966_0001683168-23...,. Risk Factors. An investment in our securitie...,Competition Risk
4,20230217_10-K_edgar_data_871763_0000950170-23-...,. Ri sk Factors FORWARD-LOOKING STATEMENTS Thi...,Ethical Risk


In [29]:
df2.tail()

,filename,Item1A,category
5,20230316_10-K_edgar_data_320121_0000320121-23-...,. Risk Factors In your evaluation of the Compa...,Ethical Risk
6,20230301_10-K_edgar_data_1725057_0000950170-23...,. R isk Factors. Risks Related to Our Business...,Ethical Risk
7,20230222_10-K_edgar_data_1788882_0001628280-23...,. Risk Factors The following are certain risk ...,Ethical Risk
8,20230227_10-K_edgar_data_1818383_0001818383-23...,. Risk Factors. Risks related to our business ...,Ethical Risk
9,20230228_10-K_edgar_data_712537_0000712537-23-...,. Risk Factors An investment in our common sto...,Ethical Risk


In [31]:
RiskFactor1 = df2.iloc[1]['Item1A']
print(RiskFactor1)

category1 = df2.iloc[1]['category']
print(category1)

. Risk Factors. Our business, prospects, financial condition, operating results and the trading price of our common stock could be materially adversely affected by a variety of risks and uncertainties, including those described below, as well as other risks not currently known to us or that are currently considered immaterial. In assessing these risks, you should also refer to the other information contained in this Annual Report on Form -K, including our consolidated financial statements and related notes. Our principal risks include risks associated with: our ability to manage our rapid growth; the challenging macroeconomic environment, including its impact on our customers' purchasing decisions on products and services which are more discretionary in nature and the related demand for our solutions relative to our expectations; the length, cost and unpredictability of our sales cycle; the development of and changes to the market for our solutions compared to our expectations; our abi